In [1]:
import timeit

import numpy as np

import pyvinecopulib as pv

# TODO: Check TLL because compiler might have large impact on performance there

# Set parameters
# n = 1000
# d = 10
n = 1000
d = 5


# Generate data
x = np.random.normal(size=n).reshape(n, 1) * np.ones(
  (n, d)
) + 0.5 * np.random.normal(size=(n, d))

# Convert data to pseudo-observations
u = pv.to_pseudo_obs(x)

controls_itau = pv.FitControlsVinecop(family_set=pv.itau, num_threads=1)
controls_itau_par_method = pv.FitControlsVinecop(
  family_set=pv.itau, parametric_method="itau", num_threads=1
)
controls_tll = pv.FitControlsVinecop(family_set=[pv.tll], num_threads=1)


# Define two different configurations of the `vinecop` function
def vinecop_itau():
  return pv.Vinecop(u, controls=controls_itau)


def vinecop_itau_par_method():
  return pv.Vinecop(u, controls=controls_itau_par_method)


def vinecop_tll():
  return pv.Vinecop(u, controls=controls_tll)


# Benchmark the functions
results = {}
for func in [vinecop_itau, vinecop_itau_par_method, vinecop_tll]:
  result = timeit.repeat(lambda: func(), globals=globals(), repeat=10, number=1)
  results[func.__name__] = result


def print_results(results, name):
  print(
    f"{name}: min={min(results)*1000:.6f}, mean={np.mean(results)*1000:.6f}, max={max(results)*1000:.6f}"
  )


# Display benchmark results
print("Unit: milliseconds:")
for name, result in results.items():
  print_results(result, name)


Unit: milliseconds:
vinecop_itau: min=1260.331131, mean=1282.708409, max=1302.413015
vinecop_itau_par_method: min=143.696779, mean=148.782970, max=164.466203
vinecop_tll: min=260.523231, mean=266.926395, max=272.306985
